# Load the data in the folder

In [ ]:
import numpy as np
import glob



Folder='./'
DataNameBasis='/*Data'

File=glob.glob(Folder+DataNameBasis+"*txt")
data_temp=np.loadtxt(File[0],skiprows=1)
ypos=data_temp[:,5]
nb_angle=len(ypos)
Data_numpyS1=data_temp[:,0]
Data_numpyS1_std=data_temp[:,1]
Data_numpyS2=data_temp[:,2]
Data_numpyS2_std=data_temp[:,3]

# Plot the Raw data

In [ ]:
import matplotlib as mat
import matplotlib.pyplot as plt


cm = 1/2.54
fig=plt.figure(figsize=(7*cm,5*cm))
linewidth=0.5

ax0 =plt.subplot(1,1,1)



mat.rcParams.update({'font.size':12,'font.family':'sans-serif','font.sans-serif':['Arial'],
                     'xtick.labelsize':9,'ytick.labelsize':9,'figure.dpi':300,'savefig.dpi':300})



ax0.plot(ypos,(Data_numpyS1-Data_numpyS2)/np.max(Data_numpyS1-Data_numpyS2))

ax0.fill_between(ypos, ((Data_numpyS1-Data_numpyS2)-(Data_numpyS1_std+Data_numpyS2_std))/np.max(Data_numpyS1-Data_numpyS2),
                  ((Data_numpyS1-Data_numpyS2)+(Data_numpyS1_std+Data_numpyS2_std))/np.max(Data_numpyS1-Data_numpyS2), alpha=0.5)






ax0.set_xlabel('y [mm]')
ax0.set_ylabel('Normalised voltage[V]', wrap=True)

plt.tight_layout()
plt.savefig('RawData.png')